In [7]:
import socket
import random
local_host = '172.30.49.217' # Replace 'localhost' with your IP address， port is 65521

In [8]:
def soccer_panalty_game(player_action, opponent_action):
    if player_action == 'left':
        m = 0
    else:
        m = 2
    
    if opponent_action == 'left':
        n = 0
    else:
        n = 1
    matrix = [0.58,0.95,0.93,0.70]
    random_number = random.random()
    if matrix[m+n] > random_number:
        return "Goal scored!"
    else:
        return "Goalie saves the shot!"        

In [9]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((local_host, 65521)) 
server_socket.listen(1)

server_socket.settimeout(10.0)  # Set a timeout of 10 seconds

print('Waiting for a connection...')
try:
    client_socket, client_address = server_socket.accept()
except socket.timeout:
    print('Connection request timed out.')
else:
    print('Connected to:', client_address)

    while True:
        # Randomly assign roles each game
        roles = ['goalie', 'player']
        random.shuffle(roles)
        server_role, client_role = roles
        client_socket.sendall(client_role.encode())
        print('You are the', server_role)
        client_action = client_socket.recv(1024).decode()
        if not client_action or client_action == 'quit':
            print('Client ended the game.' if client_action == 'quit' else '')
            break
        server_action = input("Enter your move (left or right) or 'quit' to end the game: ")
        if server_action == 'quit':
            print("You ended the game.")
            client_socket.sendall('quit'.encode())
            break
        server_action = input("Enter your move (left or right): ")
        result = soccer_panalty_game(server_action, client_action) if server_role == 'goalie' else soccer_panalty_game(client_action, server_action)
        message = f'Player chose {client_action}. Goalie chose {server_action}. {result}. As the {server_role}, you {"win" if "Goal scored" in result and server_role=="player" or "Goalie saves the shot" in result and server_role=="goalie" else "lose"}.'
        message_client = f'Player chose {client_action}. Goalie chose {server_action}. {result}. As the {client_role}, you {"lose" if "Goal scored" in result and server_role=="player" or "Goalie saves the shot" in result and server_role=="goalie" else "win"}.'
        print(message)
        client_socket.sendall(message_client.encode())
        
    client_socket.close()

Waiting for a connection...
Connected to: ('172.30.49.217', 52411)
You are the goalie
Enter your move (left or right) or 'quit' to end the game: quit
You ended the game.
